In [1]:
import tushare as ts
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import pandas as pd
import numpy as np
import baostock as bao
from baostock.data.resultset import ResultData
import baostock.common.contants as bao_consts
from pandas import Timestamp
from sz.stock_data.toolbox.data_provider import ts_code
from datetime import date, datetime, timedelta

ts.set_token('f96b1eeee9c8fddd357f2299cdedc1c88b2bb2a30ae1f772cf810dea')
ts_api = ts.pro_api()

blg = bao.login()
print('baostock login => error_code: %s error_msg: %s\n' %
      (blg.error_code, blg.error_msg))

from sz.stock_data.stock_data import StockData
StockData().setup(data_dir = '/Volumes/USBDATA/stock_data')


login success!
baostock login => error_code: 0 error_msg: success



/Users/kk/miniconda3/envs/quant/lib/python3.7/site-packages/tushare/util/upass.py:28: FutureWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated



In [2]:
df = StockData().trade_calendar.dataframe
df_trade = df[df['is_open'] == True]

In [3]:
df_trade.head(10)


,cal_date,is_open,pretrade_date
cal_date,,,
2000-01-04,2000-01-04,True,1999-12-30
2000-01-05,2000-01-05,True,2000-01-04
2000-01-06,2000-01-06,True,2000-01-05
2000-01-07,2000-01-07,True,2000-01-06
2000-01-10,2000-01-10,True,2000-01-07
2000-01-11,2000-01-11,True,2000-01-10
2000-01-12,2000-01-12,True,2000-01-11
2000-01-13,2000-01-13,True,2000-01-12
2000-01-14,2000-01-14,True,2000-01-13


In [5]:
StockData().trade_calendar.next_n_trade_day(datetime.strptime('2019-12-30', '%Y-%m-%d').date(), 5, date.today())

datetime.date(2019, 9, 20)

In [18]:
df_5min = bao.query_history_k_data_plus(
    code = "sh.600000",
    fields = "date,time,code,open,high,low,close,volume,amount,adjustflag",
    start_date='2019-09-19', 
    end_date='2019-09-20',
    frequency="5", 
    adjustflag="3"
).get_data()

df_5min.head(5)


,date,time,code,open,high,low,close,volume,amount,adjustflag
0,2019-09-19,20190919093500000,sh.600000,12.0000,12.0000,11.9600,11.9700,1362500,16329865.0000,3
1,2019-09-19,20190919094000000,sh.600000,11.9700,11.9800,11.9400,11.9800,869273,10401526.0000,3
2,2019-09-19,20190919094500000,sh.600000,11.9800,11.9900,11.9500,11.9600,426343,5103490.0000,3
3,2019-09-19,20190919095000000,sh.600000,11.9600,11.9700,11.9300,11.9400,456457,5454719.0000,3
4,2019-09-19,20190919095500000,sh.600000,11.9300,11.9600,11.9200,11.9500,487715,5820830.0000,3


In [12]:
df_5min.dtypes

date          object
time          object
code          object
open          object
high          object
low           object
close         object
volume        object
amount        object
adjustflag    object
dtype: object

In [19]:
df_5min['date'] = pd.to_datetime(df_5min['date'], format = '%Y-%m-%d')
df_5min['time'] = df_5min['time'].apply(lambda x: pd.to_datetime(x[:-3], format = '%Y%m%d%H%M%S'))
df_5min['open'] = df_5min['open'].astype(np.float64)
df_5min['high'] = df_5min['high'].astype(np.float64)
df_5min['low'] = df_5min['low'].astype(np.float64)
df_5min['close'] = df_5min['close'].astype(np.float64)

df_5min['volume'] = df_5min['volume'].astype(np.float64)
df_5min['amount'] = df_5min['amount'].astype(np.float64)
df_5min['code'] = df_5min['code'].apply(lambda x: ts_code(x))

df_5min.head(5)

,date,time,code,open,high,low,close,volume,amount,adjustflag
0,2019-09-19,2019-09-19 09:35:00,600000.SH,12.00,12.00,11.96,11.97,1362500.0,16329865.0,3
1,2019-09-19,2019-09-19 09:40:00,600000.SH,11.97,11.98,11.94,11.98,869273.0,10401526.0,3
2,2019-09-19,2019-09-19 09:45:00,600000.SH,11.98,11.99,11.95,11.96,426343.0,5103490.0,3
3,2019-09-19,2019-09-19 09:50:00,600000.SH,11.96,11.97,11.93,11.94,456457.0,5454719.0,3
4,2019-09-19,2019-09-19 09:55:00,600000.SH,11.93,11.96,11.92,11.95,487715.0,5820830.0,3


In [20]:
df_5min.dtypes


date          datetime64[ns]
time          datetime64[ns]
code                  object
open                 float64
high                 float64
low                  float64
close                float64
volume               float64
amount               float64
adjustflag            object
dtype: object

In [25]:
today = date.today()
print('today is: %s' % today)

today is: 2019-09-21
